# Dictionary Encoding & Run-End Encoding

> **Level:** Intermediate  
> **Spec:** [Dictionary-encoded layout](https://arrow.apache.org/docs/format/Columnar.html#dictionary-encoded-layout) · [Run-end encoded layout](https://arrow.apache.org/docs/format/Columnar.html#run-end-encoded-layout)  
> **PyArrow docs:** [Dictionary arrays](https://arrow.apache.org/docs/python/data.html#dictionary-arrays)

## What you will learn

1. Dictionary encoding: indices + dictionary ➡️ memory savings for low-cardinality data
2. Inspecting the indices and dictionary arrays
3. Run-End Encoding (REE): run_ends + values for long runs
5. Comparison: Dictionary vs REE vs Plain

In [ ]:
import pyarrow as pa
import numpy as np

---
## 1. Dictionary Encoding

> **Spec:** [Dictionary-encoded layout](https://arrow.apache.org/docs/format/Columnar.html#dictionary-encoded-layout)

Instead of storing `["cat", "dog", "cat", "cat", "dog"]`, store:
- **dictionary**: `["cat", "dog"]`  (unique values)
- **indices**: `[0, 1, 0, 0, 1]`  (int8/int16/int32)

For low-cardinality columns (e.g. gender, country code), this can reduce memory by 10–100×.

In [ ]:
raw = pa.array(['Washington', 'Marseille', 'Washington', 'Washington', 'Marseille'] * 100_000)
dict_arr = raw.dictionary_encode()

print('Type           :', dict_arr.type)
print('Dictionary     :', dict_arr.dictionary.to_pylist())
print('Indices[:30]    :', dict_arr.indices[:30].to_pylist())

raw_mem  = raw.nbytes
dict_mem = dict_arr.nbytes
print(f'\nRaw memory     : {raw_mem:>10,} bytes')
print(f'Dict memory    : {dict_mem:>10,} bytes')
print(f'Compression    : {raw_mem/dict_mem:.1f}x')

Type           : dictionary<values=string, indices=int32, ordered=0>
Dictionary     : ['Washington', 'Marseille']
Indices[:30]    : [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]

Raw memory     :  6,800,000 bytes
Dict memory    :  2,000,027 bytes
Compression    : 3.4×


---
## 2. Run-End Encoding (REE)

> **Spec:** [Run-end encoded layout](https://arrow.apache.org/docs/format/Columnar.html#run-end-encoded-layout)

REE stores **run_ends** (cumulative end positions) and **values**.  
For `[1.0, 1.0, 1.0, 2.0, 2.0]` ➡️  `run_ends=[3,5]`, `values=[1.0, 2.0]`.

Best for: data with long monotone runs (sensor readings, compressed logs).

In [ ]:
# Build a REE array: [Ax1000, Bx2000, Cx500]
ree_type = pa.run_end_encoded(pa.int32(), pa.utf8())
run_ends = pa.array([1000, 3000, 3500], type=pa.int32())
values   = pa.array(['A', 'B', 'C'],   type=pa.utf8())
ree = pa.RunEndEncodedArray.from_buffers(ree_type, 3500, [], children=[run_ends, values])

print('Type          :', ree.type)
print('Logical length:', len(ree))
print('Run ends      :', ree.run_ends.to_pylist())
print('Values        :', ree.values.to_pylist())
print('Memory        :', ree.nbytes, 'bytes (vs', 3500, 'bytes plain)')
print('Compression   : x', (3500)/ree.nbytes)

for idx in [0, 999, 1000, 2999, 3000, 3499]:
    print(f'  ree[{idx}] = {ree[idx]}')

Type          : run_end_encoded<run_ends: int32, values: string>
Logical length: 3500
Run ends      : [1000, 3000, 3500]
Values        : ['A', 'B', 'C']
Memory        : 27 bytes (vs 3500 bytes plain)
Compression    : x 129.62962962962962


---
## 3. Dictionary vs REE vs Plain

| Encoding | Best for | Random access | Sorted required? |
|----------|----------|--------------|------------------|
| Plain | Unique / sparse data | O(1) | No |
| Dictionary | Low-cardinality, any order | O(1) | No |
| REE | Long monotone runs | O(log k) where k=#runs | No |

In [ ]:
N = 1_000_000
categories = ['mon','tue','wed','thu','fri','sat','sun']
rng = np.random.default_rng(0)
data = [categories[i] for i in rng.integers(0,7,N)]

plain = pa.array(data, type=pa.utf8())
dicts = plain.dictionary_encode()

print(f'Plain       : {plain.nbytes:>12,} bytes')
print(f'Dictionary  : {dicts.nbytes:>12,} bytes  ({plain.nbytes/dicts.nbytes:.1f}x smaller)')

Plain       :    7,000,000 bytes
Dictionary  :    4,000,049 bytes  (1.7× smaller)


---
## Summary

- **Dictionary**: indices array + dictionary array: ideal for repeated categorical values
- **REE**: run_ends (int32) + values: ideal for long runs
- Both are first-class Arrow types: IPC transmits them faithfully without decoding

**Next ⏭️** [IPC](05_ipc_streaming_file.ipynb): wire format bytes, IPC streaming vs file